<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Model-fitting" data-toc-modified-id="Model-fitting-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Model fitting</a></span></li><li><span><a href="#Prior-distribution" data-toc-modified-id="Prior-distribution-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Prior distribution</a></span></li><li><span><a href="#Logistic-fits" data-toc-modified-id="Logistic-fits-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Logistic fits</a></span></li><li><span><a href="#Threshold---Gain-plot" data-toc-modified-id="Threshold---Gain-plot-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Threshold - Gain plot</a></span></li><li><span><a href="#Threshold---Asymmetry-plot" data-toc-modified-id="Threshold---Asymmetry-plot-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Threshold - Asymmetry plot</a></span></li><li><span><a href="#slope-vs-density" data-toc-modified-id="slope-vs-density-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>slope vs density</a></span></li><li><span><a href="#plot-half-ot-the-gain-and-r-star" data-toc-modified-id="plot-half-ot-the-gain-and-r-star-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>plot half ot the gain and r-star</a></span></li><li><span><a href="#thresholds-&amp;-density" data-toc-modified-id="thresholds-&amp;-density-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>thresholds &amp; density</a></span></li></ul></div>

# Model fitting

```
$ python fitting_models.py
```



# Prior distribution

In [1]:
import pickle as pkl
import numpy as np
import scipy.io as sio
import scipy
from scipy import stats
from scipy.optimize import curve_fit, minimize
import matplotlib.pyplot as plt
from fitting_models import value_efficient_coding_moment, fitting_model_model
from matplotlib.path import Path
from matplotlib.patches import PathPatch

savedir = './res_fit_alpha_fit/'

In [2]:
# curve
def sigmoid_func(x, a, b, c):
    return b / (1 + np.exp(-(x - c) * a))

# get first derivative of the sigmoid function
def first_derivative_sigmoid_func(x, a, b, c):
    return a*b*np.exp(-a*(x-c))/ (1 + np.exp(-a*(x-c)))**2

In [ ]:
# load fitted alpha
x_opt = []
LSE = 99999
model_name = ''
LSE_s = []
x_opt_s = []
model_name_s = []

for ii in range(1000):
    try:
        with open(savedir + 'res_fit_apprx_freealpha_freebeta_lognormal{0}.pkl'.format(ii), 'rb') as f:
            data_1 = pkl.load(f)

        LSE_s.append(data_1['res']['fun'])
        x_opt_s.append(data_1['res']['x'])
        if LSE > data_1['res']['fun']:
            LSE = data_1['res']['fun']
            x_opt = data_1['res']['x']
    except:
        ''

idxxx = np.argsort(LSE_s)
xss = np.array(x_opt_s)
id = np.argmin(LSE_s)

print(LSE_s[id]) # loss
print(xss[id]) # parameter fitted

In [ ]:
# load sigmoid fit results
# read './measured_neurons/CurveFit.ipynb' for more info

N_neurons = 39
R_t = 245.41
slope_scale = 5.07

ec = value_efficient_coding_moment(
    N_neurons=N_neurons, R_t=R_t, X_OPT_ALPH=xss[id][0], slope_scale=slope_scale)

ec_simple = value_efficient_coding_moment(
    N_neurons=N_neurons, R_t=R_t, X_OPT_ALPH=xss[id][0], slope_scale=slope_scale, simpler=True)
ec_simple.replace_with_pseudo()

dir_measured_neurons = 'measured_neurons/'

NDAT = sio.loadmat(dir_measured_neurons + 'data_max.mat')['dat']
data = sio.loadmat(dir_measured_neurons + 'curve_fit_parameters.mat')
# "curve_fit_parameters.mat" is the file that contains logistic fits of neurons
# read './measured_neurons/CurveFit.ipynb' for more info

indices = np.setdiff1d(np.linspace(0, 39, 40).astype(np.int16), 19) # except one neuron
param_set = [data['ps_lcb'][indices], data['ps'][indices], data['ps_ucb'][indices]]
midpoints = data['ps'][indices,2]

In [ ]:
%matplotlib inline 
def log_kde(x, y, sigma, x_sample):
    """ calculates a log-normal based kernel density estimate
    based on samples at x with weights/probabilities y evaluated at x_sample"""
    # mean = exp(mu + sigma^2/2) != mtrue
    # -> mu = log(mtrue) - sigma^2/2
    mu = np.log(x) - (sigma ** 2 / 2)
    y_sample = np.zeros_like(x_sample)
    for mu_i, y_i in zip(mu, y):
        y_sample += (np.exp(-((np.log(x_sample) - mu_i) ** 2) / 2 / (sigma ** 2))
                     / np.sqrt(2 * np.pi) / sigma / x_sample) * y_i
    return y_sample

ys = log_kde(midpoints, np.ones(39)/39, .6, ec.x)
ys *= np.sum(ec.p_prior) / np.sum(ys)

fig,ax = plt.subplots(1,1)

# Reward distribution
plt.plot(ec.x, ec.p_prior, color='#999999')
# Efficient code density
plt.plot(ec.x, ec.d_x, color='k')
# kde estimate of real neurons
plt.plot(ec.x, ys,color='#9ebcda')

# individual neurons
ax.vlines(midpoints, 0, .012*np.ones(midpoints.shape),colors='#9ebcda')

# Decoration
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xlim(0, 25)
ax.set_ylim(0, .22)
ax.set_xticks([0,10,20])
fig.set_figwidth(6.2)
fig.set_figheight(2)
plt.legend(['reward density', 'efficient code', 'measured neurons'])

plt.savefig('./figures/midpoints.png',bbox_inches = 'tight')
plt.savefig('./figures/midpoints.pdf',bbox_inches = 'tight')
plt.show()

# Logistic fits

As mentioned, "curve_fit_parameters.mat"  contains result of logistic fits of neurons.

Read './measured_neurons/CurveFit.ipynb' for more info

In [ ]:
# verify r_max_hat = 245.40 and get r_max's confidence interval
n_stds = 2

# calculate r_max_hat
neurons_all_ = []
for i in range(len(ec.neurons_)):
    neurons_all_.append(sigmoid_func(
        ec.x, *data['ps'][indices][i, :]))

r_max_n = [
    np.sum(np.array(neurons_all_[i]) * ec.p_prior * ec._x_gap)
    for i in range(len(neurons_all_))]
r_max_hat = np.sum(r_max_n)
print(r_max_hat) # 245.41

r_max_std = np.sqrt(N_neurons) * np.std(r_max_n)
r_max_CI_std = np.array([r_max_hat - 2 * r_max_std, r_max_hat + 2 * r_max_std])
print('min {}, max {}'.format(min(r_max_CI_std),max(r_max_CI_std)))


In [ ]:
ps_bootstrap = sio.loadmat('./measured_neurons/curve_fit_bootstrap.mat')['ps']
ps_bootstrap = ps_bootstrap[indices,:,:]
# calculate r_max's confidence interval
neurons_all = []
for j in range(ps_bootstrap.shape[1]):
    neurons_all_ = []
    for i in range(len(ec.neurons_)):
        neurons_all_.append(sigmoid_func(
            ec_simple.x, *ps_bootstrap[i, j, :]))
    neurons_all.append(np.sum(np.array(neurons_all_) * ec_simple.p_prior * ec_simple._x_gap, axis=1))
neurons_all = np.array(neurons_all)
r_max_CI_dat = [np.percentile(neurons_all, 2.5), np.percentile(neurons_all, 97.5)]


sampled_neurons_idx = np.random.randint(N_neurons, size=neurons_all.shape)
sampled_neurons = np.array([
    neu[sampled_neurons_idx[i]]
    for i, neu in enumerate(np.array(neurons_all))
])
np.sum(sampled_neurons, axis=1)
r_max_CI = [np.percentile(np.sum(sampled_neurons, axis=1), 2.5),
            np.percentile(np.sum(sampled_neurons, axis=1), 97.5)]

# Threshold - Gain plot

In [ ]:
# get asymmatric slopes 
# increase num_samples to int(1e4) 
tf, quantiles_constant, thresholds_constant, alphas, xs, ys = ec.plot_approximate_kinky_fromsim_fitting_only_raw_rstar(
    r_star=xss[id][1], num_samples=int(1e4))

In [ ]:
# load details from Dabneys paper. refer to `./measured_neurons/dabney_matlab/`
fig5 = sio.loadmat("./measured_neurons/dabney_matlab/dabney_fit.mat")
fig5_betas = sio.loadmat("./measured_neurons/dabney_matlab/dabney_utility_fit.mat")
zero_crossings = fig5['zeroCrossings_all'][:, 0]
scaleFactNeg_all = fig5['scaleFactNeg_all'][:, 0]
scaleFactPos_all = fig5['scaleFactPos_all'][:, 0]
asymM_all = fig5['asymM_all'][:, 0]
ZC_true_label = fig5['utilityAxis'].squeeze()
ZC_estimator = lambda x: fig5_betas["betas"][0, 0] + fig5_betas["betas"][1, 0] * x
idx_to_maintain = np.where((scaleFactNeg_all * scaleFactPos_all) > 0)[0]
asymM_all = asymM_all[idx_to_maintain]
asymM_all_save = asymM_all.copy()
idx_sorted = np.argsort(asymM_all)
asymM_all = asymM_all[idx_sorted]
estimated_ = np.array(ec.get_quantiles_RPs(asymM_all))
zero_crossings_ = fig5['zeroCrossings_all'][:, 0]
zero_crossings_ = zero_crossings_[idx_to_maintain]
zero_crossings_ = zero_crossings_[idx_sorted]
zero_crossings_estimated = ZC_estimator(zero_crossings_) # estimated thresholds

In [ ]:
# non linear fit + bootstrap
%matplotlib inline
fig4, ax4 = plt.subplots(1, 1)
idx_sorted_ = np.argsort(asymM_all_save)

hires_x = np.linspace(0, 15, 1000)
func = lambda x, a, b: a*(x) + b
# best_fit_ab, covar = curve_fit(func, NDAT['ZC'][0,0].squeeze(), param_set[1][:,1],
#                                absolute_sigma = True)
best_fit_ab, covar = curve_fit(func, zero_crossings_estimated, param_set[1][:,1][idx_sorted_],
                               absolute_sigma = True)

num_sim = int(5e3)
ps = []
for i in range(num_sim):
    i_sample = np.random.choice(np.linspace(0,len(zero_crossings_estimated)-1,len(zero_crossings_estimated),dtype=np.int16),len(zero_crossings_estimated))
    while True:
        try:
            fit_ab, covar = curve_fit(func, zero_crossings_estimated[i_sample], param_set[1][:,1][idx_sorted_][i_sample],
                                       absolute_sigma = True, p0=[1,1], maxfev=int(1e4))
        except:
            print('redo it with different random seed.')
        break
        
    ps.append(fit_ab)
    
scipy.stats.pearsonr(zero_crossings_estimated, param_set[1][:,1][idx_sorted_])

y_RRs = []
for count, R in enumerate(r_max_CI):
    ec = value_efficient_coding_moment(
        N_neurons=N_neurons, R_t=R, X_OPT_ALPH=xss[id][0], slope_scale=5.07)
    ec.replace_with_pseudo()
    y_RRs.append(ec.g_x_pseudo)

y_RRs = np.asarray(y_RRs)
lower = y_RRs[0]
upper = y_RRs[1]
ax4.fill_between(ec.x_inf, lower, upper,
                 color = '#9ebcda', alpha = 1)    
# ax4.plot(ec.x_inf, ec.g_x_pseudo, '-', linewidth=1,
#              c='#9ebcda',alpha=1)

ax4.set_xticks([.1, .3, 1.2, 2.5, 5, 10, 20])
ax4.scatter(zero_crossings_estimated, param_set[1][:, 1][idx_sorted_], s=10, c=[0, 0, 0])
ysample = np.asarray([func(hires_x, *pi) for pi in ps])
lower = np.percentile(ysample, 5, axis=0)
upper = np.percentile(ysample, 95, axis=0)

ax4.plot(hires_x, func(hires_x, *best_fit_ab), 'black')
ax4.fill_between(hires_x, lower, upper,
                 color = 'black', alpha = 0.15)

ax4.set_xlim(0, 12)
ax4.set_ylim(0, 40)
fig4.set_figwidth(4.5)
fig4.set_figheight(4.5)
ax4.spines['right'].set_visible(False)
ax4.spines['top'].set_visible(False)

fig4.savefig(
    './figures/threshold-gain2-boot.png')
fig4.savefig(
    './figures/threshold-gain2-boot.pdf')
# for illustration
ax4.spines['left'].set_visible(False)
ax4.spines['bottom'].set_visible(False)
ax4.set_xticks([])
ax4.set_yticks([])
fig4.savefig(
    './figures/threshold-gain2-boot3.pdf')
fig4.savefig(
    './figures/threshold-gain2-boot3.png')


# Threshold - Asymmetry plot

In [ ]:
fig, ax = plt.subplots(1,1)
RPs = ec.get_quantiles_RPs(quantiles_constant)

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.scatter(zero_crossings_estimated, asymM_all, s=10, color='k')
ax.scatter(thresholds_constant, quantiles_constant, s=10, color='#9ebcda')
ax.set_xticks([0, 1.2, 2.5, 5, 10, 20])
ax.set_xlim([0, 12])
ax.set_ylim([0, 1])
plt.grid(False)
fig.set_figwidth(4.5)
fig.set_figheight(4.5)

RPSS = ec.get_quantiles_RPs(np.linspace(0,1,1000))
# ax.plot(np.linspace(0,1,1000), RPSS, '--', color=[.7,.7,.7])
ax.plot(RPSS,np.linspace(0,1,1000), '--', color=[.7,.7,.7])

fig.savefig(
    './figures/threshold-asymm.png')
fig.savefig(
    './figures/threshold-asymm.pdf')

# slope vs density

In [ ]:
dir_measured_neurons = 'measured_neurons/'

data = sio.loadmat(dir_measured_neurons + 'curve_fit_parameters.mat')


In [ ]:
_slopes = []
_thresholds = []
for i in range(len(neurons_all_)):
    idx = np.argmin(np.abs(neurons_all_[i] - xss[id][1]))
    _thresholds.append(ec.x[idx])
    _slopes.append(first_derivative_sigmoid_func(ec.x[idx],*data['ps'][indices][i, :]))
print(_thresholds)
print(_slopes)

In [ ]:
data['ps'][indices]

In [ ]:
# fitting to sigmoid_func
num_samples = int(100)

idx_samples = np.random.choice(
    np.linspace(0,len(ec.p_prior)-1, len(ec.p_prior)),
    size=(num_samples),
    p=ec.p_prior*ec._x_gap).astype(np.int16)
x_samples = ec.x[idx_samples]

np.random.seed(1000)

neurons_sigfit = []
neuorns_ps = []
for i in range(len(ec.neurons_)):
    # one poisson sample from firing rate
    neuron_samples = [np.random.poisson(ec.neurons_[i][sample]) for sample in idx_samples]
    popt, pcov = curve_fit(
        sigmoid_func,
        x_samples,
        neuron_samples,
        p0=[1, 1, 1],
        bounds=([0, 0, 0], [np.inf, np.inf, np.inf]))
    neurons_sigfit.append(sigmoid_func(ec.x, *popt))
    neuorns_ps.append(popt)

In [ ]:
_thresholds_constant = []
_slopes_constant = []
for i in range(len(neurons_all_)):
    _thresholds_constant.append(thresholds_constant[i])
    idx = np.argmin(np.abs(ec.x - thresholds_constant[i]))
    _slopes_constant.append(first_derivative_sigmoid_func(ec.x[idx],*neuorns_ps[i]))

In [ ]:
%matplotlib inline
# scatter plot between _thresholds and _slopes
fig, ax = plt.subplots(1,2)
# plot fitted line for the scatter plot

# hires_x = np.linspace(0, 15, 1000)
# func = lambda x, a, b: a*(x) + b
# ysample = np.asarray([func(hires_x, *pi) for pi in ps])
# lower = np.percentile(ysample, 5, axis=0)
# upper = np.percentile(ysample, 95, axis=0)
# ax4.fill_between(hires_x, lower, upper,
#                  color = 'black', alpha = 0.15)

func = lambda x, slope, intercept: slope*(x) + intercept
num_sim = int(5e3)
ps = []
for i in range(num_sim):
    i_sample = np.random.choice(np.linspace(0,len(zero_crossings_estimated)-1,len(zero_crossings_estimated),dtype=np.int16),len(zero_crossings_estimated))
    while True:
        try:
            # fit the line for _thresholds and _slopes
            slope, intercept, r_value, p_value, std_err = stats.linregress(np.array(_thresholds_constant)[i_sample], np.array(_slopes_constant)[i_sample])
        except:
            print('redo it with different random seed.')
        break

    ps.append([slope, intercept])

slope, intercept, r_value, p_value, std_err = stats.linregress(_thresholds, _slopes)
ax[0].plot([0, 12], [intercept, intercept + slope * 12], 'k', lw=2)
# bow tie shaped shaded plot for the 90% confidence interval
ysample = np.asarray([func(np.linspace(0,12,100), *pi) for pi in ps])
lower = np.percentile(ysample, 5, axis=0)
upper = np.percentile(ysample, 95, axis=0)
ax[0].fill_between(np.linspace(0,12,100), lower, upper,
                 color = 'black', alpha = 0.15)

ps = []
for i in range(num_sim):
    i_sample = np.random.choice(np.linspace(0,len(zero_crossings_estimated)-1,len(zero_crossings_estimated),dtype=np.int16),len(zero_crossings_estimated))
    while True:
        try:
            # fit the line for _thresholds and _slopes
            slope, intercept, r_value, p_value, std_err = stats.linregress(np.array(_thresholds)[i_sample], np.array(_slopes)[i_sample])
        except:
            print('redo it with different random seed.')
        break

    ps.append([slope, intercept])
slope, intercept, r_value, p_value, std_err = stats.linregress(_thresholds_constant, _slopes_constant)
ax[1].plot([0, 12], [intercept, intercept + slope * 12], '#9ebcda', lw=2)
ysample = np.asarray([func(np.linspace(0,12,100), *pi) for pi in ps])
lower = np.percentile(ysample, 5, axis=0)
upper = np.percentile(ysample, 95, axis=0)
ax[1].fill_between(np.linspace(0,12,100), lower, upper,
                 color = '#9ebcda', alpha = 0.15)



ax[0].scatter(_thresholds, _slopes, s=10, color='k', alpha=0.4)
ax[1].scatter(_thresholds_constant, _slopes_constant, s=10, color='#9ebcda', alpha=0.4)


# plt.grid(True)
fig.set_figwidth(7)
fig.set_figheight(3)

# ax.legend(['Measured neurons','Efficient code'])

# correlation analysis for r and p
from scipy.stats import pearsonr
print(pearsonr(_thresholds, _slopes))
print(pearsonr(_thresholds_constant, _slopes_constant))

for i in range(2):
    ax[i].set_xlabel('Threshold')
    ax[i].set_ylabel('Slope')

    ax[i].spines['right'].set_visible(False)
    ax[i].spines['top'].set_visible(False)

    ax[i].set_xlim([0, 12])
    if i == 0:
        ax[i].set_ylim([0, 2.1])
        ax[i].set_yticks(np.linspace(0, 2, 5))
    else:
        ax[i].set_ylim([0, 4.2])
        ax[i].set_yticks(np.linspace(0, 4, 5))



fig.savefig(
    './figures/threshold-slope_model_sbs_diffaxis.png')
fig.savefig(
    './figures/threshold-slope_model_sbs_diffaxis.pdf')
plt.show()

# plot half gain and r-star


In [ ]:
NDAT = sio.loadmat(dir_measured_neurons + 'data_max.mat')['dat']
data = sio.loadmat(dir_measured_neurons + 'curve_fit_parameters.mat')['ps']

In [ ]:
half_gain_model = [ec.gsn[i] / 2 for i in range(len(ec.gsn))]


In [ ]:


# collect b from sigmoid _func
bs = []
for i in range(len(data)):
    bs.append(data[i][1])

# half of the gain
half_gain = np.array(bs) / 2

# remove the neuron that is not valid
half_gain = np.delete(half_gain, 19)

In [ ]:
%matplotlib inline
# plot small ticks of half gain
fig, ax = plt.subplots(1, 1)

# vlines plot of half gain
ax.vlines(half_gain, 0, .01*np.ones(39), colors='k')
ax.vlines(half_gain_model, 0, .01*np.ones(39), colors='#9ebcda')

from scipy.stats import norm
def normal_kde(x, y, sigma, x_sample):
    """calculates a normal based kernel density estimate
    based on samples at x with weights/probabilities y evaluated at x_sample"""
    y_sample = np.zeros_like(x_sample)
    for x_i, y_i in zip(x, y):
        y_sample += (norm.pdf((x_sample - x_i) / sigma) / sigma) * y_i
    return y_sample

# plot gaussian_kde of half gain
ys = normal_kde(half_gain, np.ones(39)/39, 1.4, ec.x[:8500])
plt.plot(ec.x[:8500], ys, color='k')

# plot gaussian_kde of half_gain_model
ys = normal_kde(half_gain_model, np.ones(39)/39, 1.4, ec.x[:8500])
plt.plot(ec.x[:8500], ys, color='#9ebcda')


# plot red vertical long line on 7.666667
# ax.vlines(7.666667, 0, .2, colors='r')
# and text that sais its $r^*$
# ax.text(7.6, .16, '$r^*$', color='r')

# small red arrow on the x axis that shows r^*
ax.annotate(
    '', xy=(7.666667, 0), xytext=(7.666667, -.04),
    arrowprops=dict(facecolor='red', shrink=0.05))

# text right below the arrow says its $r^*$
ax.text(6, -.055, '$r^*=7.67$', color='r')

# xticks and yticks and labels
ax.set_xticks(np.linspace(0, 25, 6))
ax.set_yticks(np.linspace(0, .2, 5))
ax.set_xlabel('Response (Hz)')
ax.set_ylabel('Density')

# set x and y lim
ax.set_xlim(0, 25)
ax.set_ylim(0, .2)


ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
fig.set_figwidth(6.2)
fig.set_figheight(2)
# write title that says it is half gain of the neurons
# plt.title('Half gain of the neurons')
plt.savefig('./figures/half_gain.png', bbox_inches='tight')
# pdf
plt.savefig('./figures/half_gain.pdf', bbox_inches='tight')

plt.show()


# thresholds & density


In [ ]:
# fitting to sigmoid_func
num_samples = int(100)
# get ec_moment.x samples from ec_moment.p_prior
idx_samples = np.random.choice(
    np.linspace(0,len(ec.p_prior)-1, len(ec.p_prior)),
    size=(num_samples),
    p=ec.p_prior*ec._x_gap
).astype(np.int16)
x_samples = ec.x[idx_samples]

np.random.seed(1000)

neurons_sigfit = []
neuorns_ps = []
for i in range(len(ec.neurons_)):
    # one poisson sample from firing rate
    neuron_samples = [np.random.poisson(ec.neurons_[i][sample]) for sample in idx_samples]
    popt, pcov = curve_fit(
        sigmoid_func,
        x_samples,
        neuron_samples, 
        p0=[1, 1, 1],
        bounds=([0, 0, 0], [np.inf, np.inf, np.inf])
    )
    neurons_sigfit.append(sigmoid_func(ec.x, *popt))
    neuorns_ps.append(popt)

In [ ]:
_thresholds_constant = []
_slopes_constant = []
for i in range(len(neurons_all_)):
    _thresholds_constant.append(thresholds_constant[i])
    idx = np.argmin(np.abs(ec.x - thresholds_constant[i]))
    _slopes_constant.append(first_derivative_sigmoid_func(ec.x[idx],*neuorns_ps[i]))

In [ ]:
midpoints_models = []

#find for each in ec_moment.neurons_ the index of the neuron that has the half of ec_moment.gsn
for i in range(len(ec.neurons_)):
    # argmin
    idx = np.argmin(np.abs(ec.neurons_[i] - ec.gsn[i] / 2))
    # append ec_moment.x
    midpoints_models.append(ec.x[idx])
midpoints_models = np.array(midpoints_models)

In [ ]:
%matplotlib inline

fig, ax = plt.subplots(1, 2, sharey=True, gridspec_kw={'wspace': 0.15})

sigma = .8


xs = np.log(ec.x[:8500])
ys = log_kde(midpoints, np.ones(39)/39, sigma, ec.x[:8500])
ax[0].plot(ec.x[:8500], ys, color='k')
xs = np.log(ec.x[:8500])
ys = log_kde(midpoints_models, np.ones(39)/39, sigma, ec.x[:8500])
ax[0].plot(ec.x[:8500], ys, color='#9ebcda')
ax[0].set_xlim(0, 25)
ax[0].set_ylim(0, .25)
ax[0].spines['right'].set_visible(False)
ax[0].spines['top'].set_visible(False)


xs = np.log(ec.x[:8500])
ys = log_kde(_thresholds, np.ones(39)/39,sigma, ec.x[:8500])
ax[1].plot(ec.x[:8500], ys, color='k')
xs = np.log(ec.x[:8500])
ys = log_kde(_thresholds_constant, np.ones(39)/39,sigma, ec.x[:8500])
ax[1].plot(ec.x[:8500], ys, color='#9ebcda')
ax[1].set_xlim(0, 25)
ax[1].set_ylim(0, .25)
ax[1].spines['right'].set_visible(False)
ax[1].spines['top'].set_visible(False)
# plt.plot(ec_moment.x[:8500], ys,color='#9ebcda')
# ax.vlines(midpoints, 0, .012*np.ones(midpoints.shape),colors='#9ebcda')
# ax.spines['right'].set_visible(False)
# ax.spines['top'].set_visible(False)

# proper size
fig.set_figwidth(6.2)
fig.set_figheight(2)
# set x and y labels
ax[0].set_xlabel('Reward (µL)')
ax[0].set_ylabel('Density')
ax[1].set_xlabel('Reward (µL)')
ax[1].set_ylabel('Density')

# set titles
ax[0].set_title('Midpoints')
ax[1].set_title('Thresholds')



plt.savefig('./figures/midpoints_thresholds.png', bbox_inches='tight')
plt.savefig('./figures/midpoints_thresholds.pdf', bbox_inches='tight')
plt.show()
